# 3th attempt - NN

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
SIZE = 10
AMOUNT_BOARDS = 1000

In [3]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [4]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [5]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 41366
reverse df sort: 41366


In [6]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  33506
len x test:  4137
len y train:  33506
len y test:  4137


In [7]:
X_train.shape

(33506, 100)

In [8]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [9]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 100)
(33506,)


In [10]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [11]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 10, 10, 1)
(33506, 1)


In [12]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [13]:
import tensorflow as tf

input_shape = (SIZE, SIZE, 1)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),  

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,249 (83.00 KB)

 Trainable params: 21,249 (83.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [15]:
history = model.fit(X_train_array, y_train_array, 
    validation_data=(X_val_array, y_val_array),
    epochs=10,
    batch_size=32)


Epoch 1/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8006 - loss: 0.4463 - val_accuracy: 0.8117 - val_loss: 0.3895
Epoch 2/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8226 - loss: 0.3661 - val_accuracy: 0.8206 - val_loss: 0.3766
Epoch 3/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8369 - loss: 0.3432 - val_accuracy: 0.8254 - val_loss: 0.3787
Epoch 4/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8562 - loss: 0.3124 - val_accuracy: 0.8297 - val_loss: 0.3777
Epoch 5/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8669 - loss: 0.2920 - val_accuracy: 0.8235 - val_loss: 0.3971
Epoch 6/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8783 - loss: 0.2768 - val_accuracy: 0.8200 - val_loss: 0.3955
Epoch 7/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8914 - loss: 0.2489 - val_accuracy: 0.8230 - val_loss: 0.4276
Epoch 8/10
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8984 - loss: 0.2366 - 

In [16]:
test_loss, test_acc = model.evaluate(X_test_array, y_test_array)
print(f"Test accuracy: {test_acc:.3f}")

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8011 - loss: 0.5272
Test accuracy: 0.807


In [17]:
evaluate_model(model, X_test_array, y_test_array)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

===== Evaluation Results =====
┌──────────────┬────────────┬────────────┐
│              │ Pred=Alive │ Pred=Dead  │
├──────────────┼────────────┼────────────┤
│ True=Alive   │        440 │        502 │
│ True=Dead    │        297 │       2898 │
└──────────────┴────────────┴────────────┘

--- Performance Metrics ---
Accuracy    : 0.807
Precision   : 0.597
Recall      : 0.467
F1-score    : 0.524
